In [1]:
%matplotlib widget
%matplotlib widget
import os
from pathlib import Path
import time
import torch
import numpy as np
import math
import gc
from functools import partial
from dataset_new_v2 import Dataset, load_dataframes_from_folder, reverse_normalization
from torch.utils.data import DataLoader
from transformer_zerostep_new_v2 import GPTConfig, GPT, warmup_cosine_lr
import argparse
import warnings
import matplotlib.pyplot as plt
import glob
import pandas as pd

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams['axes.labelsize']=14
plt.rcParams['xtick.labelsize']=11
plt.rcParams['ytick.labelsize']=11
plt.rcParams['axes.grid']=True
plt.rcParams['axes.xmargin']=0
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['legend.fontsize']=11

fig_width = 6
fig_width_big = 12

In [2]:

folder_path = '../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed/'
# folder_path = '../../../in-context-bldc-data/simulated/50_percent_with_alfa_beta_alt'


In [3]:
list_of_folders = glob.glob(os.path.join(folder_path, '*'))[:6]
print(list_of_folders)
identifiers = [list_of_folders[i].split("_")[-2][-9:] for i in range(len(list_of_folders))]
print(identifiers)

#### inertias [e-4]: I5 = 0.7, I13 = 1.4, I15 = 3.1, I9 = 5.8, I11 = 6.8, I7 = 8.8
order = [0, 4, 5, 2, 3, 1]
list_of_folders = [list_of_folders[j] for j in order]
identifiers = [identifiers[j] for j in order]
print(list_of_folders)
print(identifiers)

['../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\\inertia05_ki-0.0029-kp-3.0000', '../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\\inertia07_ki-0.0029-kp-3.0000', '../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\\inertia09_ki-0.0029-kp-3.0000', '../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\\inertia11_ki-0.0029-kp-3.0000', '../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\\inertia13_ki-0.0029-kp-3.0000', '../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\\inertia15_ki-0.0029-kp-3.0000']
['inertia05', 'inertia07', 'inertia09', 'inertia11', 'inertia13', 'inertia15']
['../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\\inertia05_ki-0.0029-kp-3.0000', '../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixed\\inertia13_ki-0.0029-kp-3.0000', '../data/CL_experiments_double_sensor_low_speed_ekf_and_meta/final_fixe

In [4]:

profile_list_meta = []
profile_list_ekf = []
profile_list_real = []

for exp_idx in range(len(list_of_folders)):
    profile_config_meta = []
    profile_config_ekf = []
    profile_config_real = []
    exp_path = list_of_folders[exp_idx]
    csv_list = glob.glob(os.path.join(exp_path, "*.csv"))
    for file in csv_list:

        df = pd.read_csv(file)
        profile_config_real.append(df["omega"].to_numpy())
        profile_config_meta.append(df["omega_meta"].to_numpy())
        profile_config_ekf.append(df["omega_ekf"].to_numpy())

    profile_list_real.append(  np.row_stack(profile_config_real)    )
    profile_list_meta.append(  np.row_stack(profile_config_meta)    )
    profile_list_ekf.append(   np.row_stack(profile_config_ekf)     )
    # print(np.row_stack(profile_config_real).shape)

# print(rmse_list_meta)


profile_full_real = np.stack(profile_list_real)
profile_full_meta = np.stack(profile_list_meta)
profile_full_ekf = np.stack(profile_list_ekf)
config_number, exp_number, exp_length = profile_full_meta.shape
print(config_number)
print(exp_number)
print(exp_length)

6
1
2001


In [19]:
avg_real_profile = profile_full_real[:,0,:].mean(axis = 0)
avg_meta_profile = profile_full_meta[:,0,:].mean(axis = 0)
avg_ekf_profile  =  profile_full_ekf[:,0,:].mean(axis = 0)

std_real_profile = profile_full_real[:,0,:].std(axis = 0)
std_meta_profile = profile_full_meta[:,0,:].std(axis = 0)
std_ekf_profile  =  profile_full_ekf[:,0,:].std(axis = 0)


fig = plt.figure(figsize=(6,3))

plt.plot(df['t'], avg_meta_profile, color="tab:orange", label=r"$\hat{\omega}_{\mathcal{F}}$")
plt.fill_between(df["t"], 
    avg_meta_profile + std_meta_profile, 
    avg_meta_profile - std_meta_profile, 
    color='tab:orange', 
    alpha=0.3
    )

plt.plot(df['t'], avg_ekf_profile, color="tab:blue", label=r"$\hat{\omega}_{EKF}$")
plt.fill_between(df["t"], 
    avg_ekf_profile + std_ekf_profile, 
    avg_ekf_profile - std_ekf_profile, 
    color='tab:blue', 
    alpha=0.3
    )


plt.plot(df['t'], avg_real_profile, color="tab:green", linestyle=(0, (5, 3)), label=r"$\omega$")
plt.fill_between(df["t"], 
    avg_real_profile + std_real_profile, 
    avg_real_profile - std_real_profile, 
    color='tab:green', 
    alpha=0.3
    )

# plt.plot(df['t'], df["r"], color="k", linestyle=(0, (8, 8)), linewidth = 1)
plt.legend()
plt.show()